In [ ]:
from watertap3.utils import watertap_setup, get_case_study, run_model 
from watertap3.utils import run_watertap3, run_model_no_print, run_and_return_model
import pandas as pd
import numpy as np
import haversine as hs

### 3. Enter case study information

For running a treatment train from the input sheet (```treatment_train_setup.csv```) enter the following information in this cell:

1. ```case_study```: name of case study to be run
2. ```scenario```: scenario for case study to be run 
3. ```desired_recovery```: default is 1 (for desired recovery of 100%), can be any number between 0-1
3. ```ro_bounds```: default is 'seawater', any other value will give brackish bounds


In [ ]:
case_study = 'big_spring'
scenario = 'baseline'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

### 4. Setup WaterTAP3 Model

In [ ]:
m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)


In [ ]:
# Read in brackish database. It is filtered for the correct columns. 
df = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/USGS_brackish_filtered.csv', index_col='unique_site_ID')

# convert USGS unit to our units: gpm to m3s, 
df.well_yield = df.well_yield * 6.30902e-5 #gpm to m3s
df.TDS_mgL = df.TDS_mgL / 1000 #mgL to kg/m3
df = df.rename(columns={'TDS_mgL': 'TDS_kgm3'})

path = 'C:/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/data'

# add state look up for elec prices and add elec price column
state_abbv_df = pd.read_csv(path + '/state_abbv.csv', index_col='abbv')
elec_price_df = pd.read_csv(path + '/industrial_electricity_costs_2020.csv', index_col='location')
df['state'] = df.state_alpha.map(state_abbv_df.us_state)
df['elec_price'] = df.state.map(elec_price_df.cost)

# filter for well of interest in the US
df = df[df.well_yield >= 0.01]
df = df[df.TDS_kgm3 >= 0.5]
df = df[df.Latitude >= 22]
df = df[df.Longitude >= -150]
df = df[df.TDS_kgm3 <= 25]

In [ ]:
len(df)

In [ ]:
# run the baseline watertap3 version of this treatment train
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

In [ ]:
m.fs.costing.LCOW.value()

In [ ]:
# unfix so that the membrane area and pressure can be optimized based on source water conditions
m.fs.reverse_osmosis.membrane_area.unfix()
m.fs.reverse_osmosis.feed.pressure.unfix()

# allow TDS levels to be qua

In [ ]:
# loop through brackish sites and calculate+save LCOW, capital, O&M, energy intensity, and recovery in a df.
capital = []
recovery = []
onm = []
lcow = []
elec_int = []
tds_out = []
flow_in_list = []
tds_in_list = []
i = 0

for site in df.index:
    print('-----------------------------')
    print('SITE NAME:', site)
    i = i + 1
    print(i/len(df.index))
           
    m.fs.big_spring_feed.flow_vol_in.fix(df.loc[site].well_yield)
    m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(df.loc[site].TDS_kgm3)
    m.fs.well_field.lift_height.fix(df.loc[site].well_depth_ft)
    
    m.fs.costing_param.electricity_price = df.loc[site].elec_price

    m = run_and_return_model(m, objective=True)
    
    flow_in_list.append(df.loc[site].well_yield)
    capital.append(m.fs.costing.capital_investment_total())
    recovery.append(m.fs.costing.system_recovery()*100)
    onm.append(m.fs.costing.operating_cost_annual())
    lcow.append(m.fs.costing.LCOW())
    elec_int.append(m.fs.costing.electricity_intensity())
    tds_out.append(m.fs.municipal_drinking.conc_mass_in[0, 'tds']())
    tds_in_list.append(df.loc[site].TDS_kgm3)
    
df['lcow'] = lcow
df['recovery'] = recovery
df['capital'] = capital
df['onm'] = onm
df['elec_int'] = elec_int
df['tds_out'] = tds_out



In [ ]:
m.fs.costing.treated_water.value

In [ ]:
df.head()

In [ ]:
# for a given brackish site, record all power plants within a certain km radius
def get_nearby_power_plants(df_in, site_in, radius):

    # read in power plant dataset
    netl_df = pd.read_csv("/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Power/NETL_ThermalPlants_Filtered.csv")
    # filter for power plants that we have data for, and that only use freshwater
    pp_df = netl_df[((netl_df.WaterType == 'Fresh') & (netl_df.WAvg != '...') & (netl_df.WAvg != '---'))]
    pp_df.WAvg = pp_df.WAvg.astype(float)
    pp_df.rename({'Longitude': 'PPLongitude', 'Latitude': 'PPLatitude'}, axis=1, inplace=True)
    
    dist_km_list = []
    pp_df_out = pd.DataFrame()
    pp_df_out_final = pd.DataFrame()

    bw_lat = df_in.loc[site_in].Latitude
    bw_lon = df_in.loc[site_in].Longitude
    loc1 = (bw_lat, bw_lon)

    for pp_index in pp_df.index: #[1]:#

        pp_lat = pp_df.loc[pp_index].PPLatitude
        pp_lon = pp_df.loc[pp_index].PPLongitude
        loc2 = (pp_lat, pp_lon)

        dist_km = hs.haversine(loc1,loc2)

        if dist_km < radius:

            dist_km_list.append(dist_km)

            pp_df_hold = pp_df[["PlantCode", "Fuel", "Cooling", "Capacity", "PPLatitude", "PPLongitude", "WAvg"]][pp_df.index == pp_index] 
                                #, "Fuel", "Cooling, "Capacity", "Latitude", "Longitude", 
           #'withd', 'cons', 'mup', 'final_with']][pp_df.index == pp_index]

            pp_df_out = pd.concat([pp_df_out,pp_df_hold], ignore_index=True)

    pp_df_out['dist_km'] = dist_km_list
    pp_df_out['site'] = np.array(site_in)

    for c_name in df.columns:
        pp_df_out[c_name] = pp_df_out.site.map(df[c_name])

        # calculate LCOW w/transportation costs

    pp_df_out_final = pd.concat([pp_df_out_final,pp_df_out])

    return pp_df_out_final
    

In [ ]:
# loop through all brackish sites, and match power plants. 
df_pp_bw_linked = pd.DataFrame()

i = 0 

for site_in in df.index:
    print('-----------------------------')
    print('SITE NAME:', site)
    i = i + 1
    print(100*i/len(df))
    
    df_site = get_nearby_power_plants(df, site_in, 50) 
    df_pp_bw_linked = pd.concat([df_pp_bw_linked,df_site])

# calculate the cooling demand met by brackish sites (one-to-one match onnly, but should calculate many to one)    
df_pp_bw_linked['withm3s'] = (df_pp_bw_linked.WAvg * 0.0037854 / 3600) * df_pp_bw_linked.Capacity
df_pp_bw_linked['metdemand'] = ((df_pp_bw_linked.recovery/100) * df_pp_bw_linked.well_yield) / df_pp_bw_linked.withm3s


In [ ]:
df_pp_bw_linked.head()

In [ ]:
df_pp_bw_linked.to_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Source to Use Scenarios/Brackish to Power/df_pp_bw_linked.csv')

In [ ]:
df_pp_bw_linked = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Source to Use Scenarios/Brackish to Power/df_pp_bw_linked.csv')

In [ ]:
path = "/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Source to Use Scenarios/Brackish to Power/"

In [ ]:
df_pp_bw_linked[df_pp_bw_linked.dist_km <= 25].to_csv(path+'df_pp_bw_linked0_25.csv')
df_pp_bw_linked[df_pp_bw_linked.dist_km > 25].to_csv(path+'df_pp_bw_linked25_50.csv')

In [ ]:
df_pp_bw_linked.keys()

In [ ]:
pp_group = df_pp_bw_linked.groupby('PlantCode')
site_group = df_pp_bw_linked.groupby('site')

In [ ]:
pp_group.head(10)

In [ ]:
pp_group = df_pp_bw_linked.set_index(["PlantCode","site"])


In [ ]:
one_pp_many_bw_cols = ["PlantCode", "PPDemand" , "BWSites", "BWAvailable", "CapitalCost" , "NewOnM"]

In [ ]:
# create code for many to one, and one to many, by using groupby function.

In [ ]:
# Group powerplants supplied by multiple brackish wells (all) - list well ids and throughput matching a powerplant
# Group brackish wells and list powerplants being supplied to - ids fraction supply met

In [ ]:
pp_group.get_group(108)

In [ ]:
# LCOW = (Annualized capital cost + total onm) / annual water product
# add annualized capital cost column to dataframe
# $/m3-km value (guess value) - 
# new total onm column = total onm + $/m3- km * km(dist between) * vol of prod in 1 year
# New LCOW = (annualized CC + new total onm)/ an.

In [ ]:
# scenarios using data from other sub tasks

In [ ]:
# brine disposal costs - could add to WT3 or do after based on brine/recovery factor with TDS at outlet

In [ ]:
# transportation costs - could add to WT3 or do after based on distance.